In [1]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

Optimal path:
Go through each tube station in the London tube system with the shortest route possible

Lenght:
X number of episodes (1 episode=1 station)

Actions:
Go straight (remaining in same tube line), Go back (remaining in same tube line), change tube line and decide to go straight or back



In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [3]:
class TubeEnv(env):
    def __init__(self):
        # Actions we can take, down, up, change
        self.action_space = Discrete(3) #3 values (0,1,2)
        # Number of stations visited
        self.observation_space = Box(low=np.array([0]), high=np.array([100])) #0 if station was already discovered
        # Set start station
        self.state = ? #initial tube station
        # Set journey length
        self.journey_length = 1000 #whatever we want
    def step(self, action):
        # Apply action
        # 0 -1 = up 
        # 1 -1 = change
        # 2 -1 = down
        self.state += action -1 
        # Reduce journey length by 1 station
        self.journey_length -= 1 
        
        # Calculate reward
        if self.state >=100: #so if you visited all the stations
            reward =1 
        else: 
            reward = -1 
        
        # Check if journey is done
        if self.journey_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset journey temperature
        self.state = ? #should we reset to the same station
        # Reset journey time
        self.journey_length = 1000 
        return self.state

SyntaxError: invalid syntax (2009089951.py, line 8)

https://github.com/aurelienbibaut/Actor_CriticPointer_Network-TSP

In [ ]:
import numpy as np


def adjacency_matrix(list_nodes):
    """
    compute the adjacency matrix
    input : list of nodes example (1,2,3,4)
    output: adjacency matrix example [[0,1,0,1],[1,0,1,0],[0,1,0,1],[1,0,1,0]]
    """

    # check min index is 0 or 1
    min_index = min(list_nodes)
    length_list = len(list_nodes)
    if min_index == 1:
        list_nodes = np.array(list_nodes) - 1

    adj_mat = np.zeros((length_list, length_list))
    for n in range(length_list - 1):
        i = list_nodes[n]
        j = list_nodes[n + 1]
        adj_mat[i][j] = 1
        adj_mat[j][i] = 1
    i = list_nodes[length_list - 1]
    j = list_nodes[0]
    adj_mat[i][j] = 1
    adj_mat[j][i] = 1
    return adj_mat


def flatten_adj_matrix(adj_matrix):
    """
    flatten a N x N into a 1 x N^2 vector
    input : N x N matrix
    output : 1 x N^2 matrix
    """
    return adj_matrix.reshape((1, -1))


def adj_matrix_from_vector(vector, number_nodes):
    """
    retrieve the adjacency matrix from a vector
    input : 1 x N^2 matrix
    output : N x N matrix
    """
    return vector.reshape((number_nodes, number_nodes))


def all_circular_permutations(solution):
    """
    compute all the circular permutation of a solution
    (list of nodes) which are also a solution.
    input : list of nodes (solution)
    output : list of circular permutations of solution
    """
    n = len(solution)
    return [[solution[i - j] for i in range(n)] for j in range(n)]


def check_optimality_supervised(solution, opt_solution):
    """
    check wether solution is cicular permutation of
    opt_solution
    input : solution and optimal solution
    output : boolean
    """
    return solution in all_circular_permutations(opt_solution)


def check_neighbor_constraint(solution_adj_matrix, node, min_index):
    """
    check whether a node in a solution
    has exactly 2 neighbors or not.
    input : solution adjacency matrix, node
    output : boolean
    """
    if min_index == 1:
        node -= 1
    return sum(solution_adj_matrix[node]) == 2


def check_subtour_constraints(list_nodes, solution_adj_matrix):
    """
    check whether the subtour constraints
    are violated or not.
    input : solution adjacency matrix, node
    output : boolean
    """
    import itertools
    # check min index is 0 or 1
    min_index = min(list_nodes)
    length_list = len(list_nodes)
    if min_index == 1:
        list_nodes = np.array(list_nodes) - 1

    subtour_sizes = range(1, length_list)
    subtours = []
    for k in subtour_sizes:
        l = list(itertools.combinations(list_nodes, k))
        subtours.append([list(ll) for ll in l])
    for k_subtours in subtours:
        for subtour in k_subtours:
            if np.sum(solution_adj_matrix[np.ix_(
                    subtour, subtour)]) - len(subtour) >= len(subtour):
                return False

    return True


def compute_cost(distance_matrix, solution_adj_matrix, tour_length):
    """
    compute the total distance of a tour
    input : distance matrix, solution adjacency matrix, tour length
    output : tour cost
    """
    cost = 0
    for i in range(tour_length):
        cost += np.sum(distance_matrix[i] * solution_adj_matrix[i])

    return cost / 2


def node_list_from_adj_matrix(adj_matrix, number_of_nodes,min_index):
    """
    return a permutation using the adjacency matrix
    input : adjacency matrix, number of nodes
    output : a permutation of a tour
    """

    visited = [0]
    for i in range(number_of_nodes - 1):
        l = list(np.where(adj_matrix[visited[i]])[0])

        next_neighbor = [x for x in l if x not in visited]
        visited.append(np.min(next_neighbor))

    if min_index:
        visited = [x + 1 for x in visited]

    return visited

def generate_n_random_point(n):
    import random
    node_list =[]
    for i in range(n):
        x_i = random.random()
        y_i = random.random()
        point=np.array([x_i,y_i])
        node_list.append(point)
    return np.array(node_list)

def compute_distance_for_list(node_list):
    d = 0
    for i in range(len(node_list)-1):
        d += np.linalg.norm(node_list[i]-node_list[i+1])
    d += np.linalg.norm(node_list[-1]-node_list[0])
    return d



In [ ]:
import numpy as np
import random
import itertools
import math

def length(x, y):
    dx = x[0] - y[0]
    dy = x[1] - y[1]
    return math.sqrt(dx*dx + dy*dy)

class TSP_env(object):
    def __init__(self, n, no_move_penalty=0, use_alternative_state=False):
        """
        current_states : contains the indexes of the selected nodes eg [1,3,4]
        nodes : contains all the nodes and their 2 dimension coordinates [n,2] dimension array eg [[0.5,0.7],[0.2,0.3]]
        n : number of nodes
        """
        self.number_nodes = n
        self.no_move_penalty = no_move_penalty
        self.num_actions = self.number_nodes

        self.use_alternative_state = use_alternative_state
        if use_alternative_state:
            self.state_shape = [4 * n]
        else:
            self.state_shape = [n]
        self.env_name = 'TSP'


    def binary_vector_state(self):
        xs = np.zeros(self.number_nodes)
        for i in self.current_state:
            xs[i] = 1
        return xs

    def get_alternative_state(self):
        current_position = np.zeros(self.number_nodes)
        if len(self.current_state) > 0:
            current_position[self.current_state[-1]] = 1
        return np.concatenate([self.nodes[:, 0],
                               self.nodes[:, 1],
                               self.binary_vector_state(),
                               current_position])

    def compute_subtour_length(self):
        subtour_length = 0
        if len(self.current_state) > 1:
            for i in range(len(self.current_state) - 1):
                subtour_length += self.weight_matrix[self.current_state[i],\
                                                     self.current_state[i + 1]]
            subtour_length += self.weight_matrix[self.current_state[i + 1],\
                                                 self.current_state[0]]
        return subtour_length

    def reset(self):
        self.current_state = []
        self.nodes = generate_n_random_point(self.number_nodes)
        self.adjacency_matrix = np.ones((self.number_nodes, self.number_nodes))
        np.fill_diagonal(self.adjacency_matrix, 0)
        x1s, x2s = self.nodes[:, 0], self.nodes[:, 1]
        self.weight_matrix = np.sqrt(np.power(np.subtract.outer(x1s, x1s), 2) + \
                                     np.power(np.subtract.outer(x2s, x2s), 2))
        np.fill_diagonal(self.weight_matrix, self.no_move_penalty)

        if not self.use_alternative_state:
            return self.binary_vector_state()
        else:
            return self.get_alternative_state()

    def accumulated_reward(self):
        return -self.compute_subtour_length()

    def at_random_solution(self, visit_once=True):
        """Pick a city at random, allowing cities to be visited more than once"""
        tour = [np.random.randint(self.number_nodes)]
        while set(tour) != set(range(self.number_nodes)):
            if visit_once:
                proposed_city = np.random.randint(self.number_nodes)
                if proposed_city not in tour:
                    tour.append(proposed_city)
            else:
                tour.append(np.random.randint(self.number_nodes))
        tour.append(tour[0])
        tour_length = 0
        for i in range(len(tour) - 1):
            tour_length += self.weight_matrix[tour[i], tour[i+1]]

        return -tour_length, tour

    def best_solution_from_now(self):
        if len(self.current_state) == 0:
            return self.optimal_solution()

        remaining_cities = list(set(range(self.number_nodes)) - set(self.current_state))
        paths = list(itertools.permutations(remaining_cities))
        tour_lengths = []
        for path in paths:
            tour_length = 0
            candidate_tour = self.current_state + list(path) + [self.current_state[0]]
            for i in range(len(candidate_tour) - 1):
                tour_length += self.weight_matrix[candidate_tour[i], candidate_tour[i + 1]]
            tour_lengths.append(tour_length)
        return -np.min(tour_lengths), paths[np.argmin(tour_lengths)]

    def optimal_solution(self):
        # calc all lengths
        points = self.nodes
        all_distances = [[length(x, y) for y in points] for x in points]
        # initial value - just distance from 0 to every other point + keep the track of edges
        A = {(frozenset([0, idx + 1]), idx + 1): (dist, [0, idx + 1]) for idx, dist in enumerate(all_distances[0][1:])}
        cnt = len(points)
        for m in range(2, cnt):
            B = {}
            for S in [frozenset(C) | {0} for C in itertools.combinations(range(1, cnt), m)]:
                for j in S - {0}:
                    B[(S, j)] = min([(A[(S - {j}, k)][0] + all_distances[k][j], A[(S - {j}, k)][1] + [j]) for k in S if
                                     k != 0 and k != j])  # this will use 0th index of tuple for ordering, the same as if key=itemgetter(0) used
            A = B
        res = min([(A[d][0] + all_distances[0][d[1]], A[d][1]) for d in iter(A)])
        return -res[0], res[1]

    def step(self, action):
        done = False

        if self.current_state == []:
            self.current_state = [action]
            added_distance = 0
        else:
            if not(action in self.current_state):
                subtour_length = self.compute_subtour_length()
                self.current_state.append(action)
                new_subtour_length = self.compute_subtour_length()
                added_distance = new_subtour_length - subtour_length
            else:
                added_distance = self.no_move_penalty

        if len(self.current_state) == self.number_nodes:
            done = True
        if not self.use_alternative_state:
            return self.binary_vector_state(), -added_distance, done
        else:
            return self.get_alternative_state(), -added_distance, done


In [ ]:
env = TSP_env

creating node network

In [ ]:
pip install plotly

In [ ]:
import plotly.graph_objects as go

import networkx as nx

G = nx.random_geometric_graph(50, 0.125)
print(G)

In [ ]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [ ]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:

characters = ['Baker Street', 'Charing Cross', 'Edgware Road (B)', 'Elephant & Castle',
 'Embankment', 'Harlesden', 'Harrow & Wealdstone', 'Kensal Green', 'Kenton',
 'Kilburn Park', 'Lambeth North', 'Maida Vale', 'North Wembley',
 'Oxford Circus', 'Paddington', 'Stonebridge Park']


edges = [('Baker Street', 'Charing Cross'),
        ('Charing Cross', 'Edgware Road (B)'),
        ('Edgware Road (B)', 'Elephant & Castle'),
        ('Elephant & Castle',
 'Embankment'),
        ('Embankment', 'Harlesden'),
        ('Harlesden', 'Harrow & Wealdstone'),
        ('Harrow & Wealdstone', 'Kensal Green'),
        ('Kensal Green', 'Kenton'),
        ('Kenton',
 'Kilburn Park'),
        ('Kilburn Park', 'Lambeth North'),
        ('Lambeth North', 'Maida Vale'),
        ('Maida Vale', 'North Wembley'),
        ('North Wembley',
 'Oxford Circus'),
        ('Oxford Circus', 'Paddington'),
        ('Paddington', 'Stonebridge Park')]

G_Bakerloo = nx.Graph()


G_Bakerloo.add_nodes_from(characters)
G_Bakerloo.add_edges_from(edges)

In [ ]:
nx.line_graph(G_Bakerloo)
nx.draw(G_Bakerloo, with_labels = True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
L = nx.line_graph(G_Bakerloo)
plt.show(L)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
G=nx.Graph()
G.add_edges_from([('A',1),(1,2),(2,3),(3,'A')])
positions = {'A':[0,0],1:[1,0],2:[1,1],3:[0,1]}
nx.draw_networkx(G,pos=positions)
